In [54]:
import numpy as np
import sys


import collections
import re

import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, LSTM, Dropout, Input
from keras.optimizers import RMSprop, Adam

from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras.utils import np_utils

import tensorflow as tf

In [2]:
print(tf.__version__)
print(keras.__version__)

1.2.1
2.0.6


In [3]:
# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [4]:
raw_text = raw_text[1:]

## Create look up tables

In [5]:
# create mapping of unique chars to integers

voc_cnt = collections.Counter(raw_text.split())
vocab = sorted(voc_cnt, key=voc_cnt.get, reverse=True)
words = sorted(list(set(raw_text.split())))
total_words = len(raw_text.split())

# Lookup tables
word_to_int = dict((c, i) for i, c in enumerate(vocab))
int_to_word = dict((i, c) for i, c in enumerate(vocab)) 

# summarize the loaded data

n_vocab_words = len(words)

print("Total Vocab: ", n_vocab_words)
print("words",total_words)
print("total number of unique words", n_vocab_words)


Total Vocab:  4938
words 26438
total number of unique words 4938


In [6]:
print(int_to_word[230])
print(word_to_int['cat'])
print(len(word_to_int))

wonder
229
4938


## Make the sequences 

This time we are making the look up tables based on words rather than characters this allows for 

In [7]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 30
dataX = []
dataY = []
split_text = raw_text.split()
for i in range(0, len(words) - seq_length, 1):
    seq_in = split_text[i:i + seq_length]
    seq_out = split_text[i + seq_length]
    dataX.append([word_to_int[word] for word in seq_in])
    dataY.append(word_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  4908


In [8]:
len(raw_text.split()) - seq_length

26408

### Lets examine some of these sequences

In [9]:
print(dataX[0])
print(dataY[0])

[280, 2977, 40, 0, 1181, 14, 9, 318, 2, 82, 21, 441, 5, 338, 61, 15, 677, 20, 0, 1634, 1, 5, 333, 164, 2, 4610, 189, 48, 3235, 4]
17


In [10]:
print("\"", ' '.join([int_to_word[value] for value in dataX[0]]), "\"")
print(int_to_word[dataY[0]])

" chapter i. down the rabbit-hole alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she "
had


In [11]:
print(dataX[1])
print(dataY[1])

[2977, 40, 0, 1181, 14, 9, 318, 2, 82, 21, 441, 5, 338, 61, 15, 677, 20, 0, 1634, 1, 5, 333, 164, 2, 4610, 189, 48, 3235, 4, 17]
1144


In [12]:
print("\"", ' '.join([int_to_word[value] for value in dataX[1]]), "\"")
print(int_to_word[dataY[1]])

" i. down the rabbit-hole alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had "
peeped


### Reshaping the sequences to become timesteps into the LSTM

In [13]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
print(X.shape)
# normalize
X = X / float(n_vocab_words)

(4908, 30, 1)


In [14]:
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [15]:
print(X[0][:10])
print(y[0])

[[  5.67031187e-02]
 [  6.02875658e-01]
 [  8.10044552e-03]
 [  0.00000000e+00]
 [  2.39165654e-01]
 [  2.83515593e-03]
 [  1.82260024e-03]
 [  6.43985419e-02]
 [  4.05022276e-04]
 [  1.66059133e-02]]
[ 0.  0.  0. ...,  0.  0.  0.]


In [27]:
inp = Input(shape=(X.shape[1], X.shape[2]))
print('our input shape is ',(X.shape[1], X.shape[2]) )

our input shape is  (30, 1)


In [38]:
x = LSTM(256)(inp)
x = Dropout(0.2)(x)
#x = LSTM(256)(x)
#x = Dropout(0.2)(x)
output = Dense(y.shape[1], activation ='softmax')(x)

In [39]:
generative_model = Model(inputs = inp, outputs=output )

In [40]:
generative_model.compile(loss='categorical_crossentropy', optimizer='adam')

In [57]:
# define the checkpoint callback
filepath="checkpoints/Word-gen-text-weights-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
reduce_LR = ReduceLROnPlateau(monitor='loss',factor = 0.9, patience=3,cooldown=2, min_lr = 0.00001)
callbacks_list = [checkpoint,reduce_LR]

In [58]:
generative_model.fit(X, y, epochs=10, batch_size=64, callbacks=callbacks_list)

Epoch 1/10
4908/4908 [==============================] - 1s - loss: 6.0797     
Epoch 2/10
4908/4908 [==============================] - 1s - loss: 6.0544     
Epoch 3/10
4908/4908 [==============================] - 1s - loss: 6.0231     
Epoch 4/10
4908/4908 [==============================] - 1s - loss: 5.9903     
Epoch 5/10
4908/4908 [==============================] - 1s - loss: 5.9519     
Epoch 6/10
4908/4908 [==============================] - 1s - loss: 5.9075     
Epoch 7/10
4908/4908 [==============================] - 1s - loss: 5.8649     
Epoch 8/10
4908/4908 [==============================] - 1s - loss: 5.8185     
Epoch 9/10
4908/4908 [==============================] - 1s - loss: 5.7735     
Epoch 10/10
4908/4908 [==============================] - 1s - loss: 5.7184     


In [59]:
generative_model.save('Text_gen_01_words_no_embedding')

In [60]:
generative_model = load_model('Text_gen_01_words_no_embedding')

In [99]:
generative_model.fit(X, y, epochs=500, batch_size=64, callbacks=callbacks_list)

Epoch 1/500
4908/4908 [==============================] - 1s - loss: 2.0955     
Epoch 2/500
4908/4908 [==============================] - 1s - loss: 2.0953     
Epoch 3/500
4908/4908 [==============================] - 1s - loss: 2.0529     
Epoch 4/500
4908/4908 [==============================] - 1s - loss: 2.0693     
Epoch 5/500
4908/4908 [==============================] - 1s - loss: 2.0202     
Epoch 6/500
4908/4908 [==============================] - 1s - loss: 2.0158     
Epoch 7/500
4908/4908 [==============================] - 1s - loss: 1.9934     
Epoch 8/500
4908/4908 [==============================] - 1s - loss: 1.9909     
Epoch 9/500
4908/4908 [==============================] - 1s - loss: 1.9767     
Epoch 10/500
4908/4908 [==============================] - 1s - loss: 1.9633     
Epoch 11/500
4908/4908 [==============================] - 1s - loss: 1.9652     
Epoch 12/500
4908/4908 [==============================] - 1s - loss: 1.9521     
Epoch 13/500
4908/4908 [=============

4908/4908 [==============================] - 1s - loss: 1.6863     
Epoch 35/500
4908/4908 [==============================] - 1s - loss: 1.6905     
Epoch 36/500
4908/4908 [==============================] - 1s - loss: 1.6738     
Epoch 37/500
4908/4908 [==============================] - 1s - loss: 1.6498     
Epoch 38/500
4908/4908 [==============================] - 1s - loss: 1.6540     
Epoch 39/500
4908/4908 [==============================] - 1s - loss: 1.6393     
Epoch 40/500
4908/4908 [==============================] - 1s - loss: 1.6225     
Epoch 41/500
4908/4908 [==============================] - 1s - loss: 1.6275     
Epoch 42/500
4908/4908 [==============================] - 1s - loss: 1.6195     
Epoch 43/500
4908/4908 [==============================] - 1s - loss: 1.6140     
Epoch 44/500
4908/4908 [==============================] - 1s - loss: 1.6248     
Epoch 45/500
4908/4908 [==============================] - 1s - loss: 1.5951     
Epoch 46/500
4908/4908 [=================

4908/4908 [==============================] - 1s - loss: 1.3843     
Epoch 70/500
4908/4908 [==============================] - 1s - loss: 1.4024     
Epoch 71/500
4908/4908 [==============================] - 1s - loss: 1.3873     
Epoch 72/500
4908/4908 [==============================] - 1s - loss: 1.3671     
Epoch 73/500
4908/4908 [==============================] - 1s - loss: 1.3653     
Epoch 74/500
4908/4908 [==============================] - 1s - loss: 1.3499     
Epoch 75/500
4908/4908 [==============================] - 1s - loss: 1.3759     
Epoch 76/500
4908/4908 [==============================] - 1s - loss: 1.3321     
Epoch 77/500
4908/4908 [==============================] - 1s - loss: 1.3202     
Epoch 78/500
4908/4908 [==============================] - 1s - loss: 1.3270     
Epoch 79/500
4908/4908 [==============================] - 1s - loss: 1.3063     
Epoch 80/500
4908/4908 [==============================] - 1s - loss: 1.3363     
Epoch 81/500
4908/4908 [=================

4908/4908 [==============================] - 1s - loss: 1.1894     
Epoch 106/500
4908/4908 [==============================] - 1s - loss: 1.1305     
Epoch 107/500
4908/4908 [==============================] - 1s - loss: 1.1009     
Epoch 108/500
4908/4908 [==============================] - 1s - loss: 1.0936     
Epoch 109/500
4908/4908 [==============================] - 1s - loss: 1.1002     
Epoch 110/500
4908/4908 [==============================] - 1s - loss: 1.0787     
Epoch 111/500
4908/4908 [==============================] - 1s - loss: 1.0880     
Epoch 112/500
4908/4908 [==============================] - 1s - loss: 1.1035     
Epoch 113/500
4908/4908 [==============================] - 1s - loss: 1.0653     
Epoch 114/500
4908/4908 [==============================] - 1s - loss: 1.1062     
Epoch 115/500
4908/4908 [==============================] - 1s - loss: 1.0817     
Epoch 116/500
4908/4908 [==============================] - 1s - loss: 1.0550     
Epoch 117/500
4908/4908 [=====

4908/4908 [==============================] - 1s - loss: 0.9150     
Epoch 143/500
4908/4908 [==============================] - 1s - loss: 0.9062     
Epoch 144/500
4908/4908 [==============================] - 1s - loss: 0.9174     
Epoch 145/500
4908/4908 [==============================] - 1s - loss: 0.9120     
Epoch 146/500
4908/4908 [==============================] - 1s - loss: 0.8838     
Epoch 147/500
4908/4908 [==============================] - 1s - loss: 0.8974     
Epoch 148/500
4908/4908 [==============================] - 1s - loss: 0.9007     
Epoch 149/500
4908/4908 [==============================] - 1s - loss: 0.9073     
Epoch 150/500
4908/4908 [==============================] - 1s - loss: 0.9138     
Epoch 151/500
4908/4908 [==============================] - 1s - loss: 0.8909     
Epoch 152/500
4908/4908 [==============================] - 1s - loss: 0.8774     
Epoch 153/500
4908/4908 [==============================] - 1s - loss: 0.8627     
Epoch 154/500
4908/4908 [=====

...

4908/4908 [==============================] - 1s - loss: 0.4759     
Epoch 299/500
4908/4908 [==============================] - 1s - loss: 0.4818     
Epoch 300/500
4908/4908 [==============================] - 1s - loss: 0.4804     
Epoch 301/500
4908/4908 [==============================] - 1s - loss: 0.4879     
Epoch 302/500
4908/4908 [==============================] - 1s - loss: 0.4891     
Epoch 303/500
4908/4908 [==============================] - 1s - loss: 0.4801     
Epoch 304/500
4908/4908 [==============================] - 1s - loss: 0.4695     
Epoch 305/500
4908/4908 [==============================] - 1s - loss: 0.4830     
Epoch 306/500
4908/4908 [==============================] - 1s - loss: 0.4702     
Epoch 307/500
4908/4908 [==============================] - 1s - loss: 0.4804     
Epoch 308/500
4908/4908 [==============================] - 1s - loss: 0.4714     
Epoch 309/500
4908/4908 [==============================] - 1s - loss: 0.4711     
Epoch 310/500
4908/4908 [=====

4908/4908 [==============================] - 1s - loss: 0.4416     
Epoch 339/500
4908/4908 [==============================] - 1s - loss: 0.4376     
Epoch 340/500
4908/4908 [==============================] - 1s - loss: 0.4373     
Epoch 341/500
4908/4908 [==============================] - 1s - loss: 0.4396     
Epoch 342/500
4908/4908 [==============================] - 1s - loss: 0.4232     
Epoch 343/500
4908/4908 [==============================] - 1s - loss: 0.4286     
Epoch 344/500
4908/4908 [==============================] - 1s - loss: 0.4414     
Epoch 345/500
4908/4908 [==============================] - 1s - loss: 0.4325     
Epoch 346/500
4908/4908 [==============================] - 1s - loss: 0.4363     
Epoch 347/500
4908/4908 [==============================] - 1s - loss: 0.4258     
Epoch 348/500
4908/4908 [==============================] - 1s - loss: 0.4299     
Epoch 349/500
4908/4908 [==============================] - 1s - loss: 0.4275     
Epoch 350/500
4908/4908 [=====

4908/4908 [==============================] - 1s - loss: 0.4031     
Epoch 381/500
4908/4908 [==============================] - 1s - loss: 0.4116     
Epoch 382/500
4908/4908 [==============================] - 1s - loss: 0.4018     
Epoch 383/500
4908/4908 [==============================] - 1s - loss: 0.4171     
Epoch 384/500
4908/4908 [==============================] - 1s - loss: 0.3952     
Epoch 385/500
4908/4908 [==============================] - 1s - loss: 0.4200     
Epoch 386/500
4908/4908 [==============================] - 1s - loss: 0.4055     
Epoch 387/500
4908/4908 [==============================] - 1s - loss: 0.3952     
Epoch 388/500
4908/4908 [==============================] - 1s - loss: 0.4041     
Epoch 389/500
4908/4908 [==============================] - 1s - loss: 0.4033     
Epoch 390/500
4908/4908 [==============================] - 1s - loss: 0.3998     
Epoch 391/500
4908/4908 [==============================] - 1s - loss: 0.4276     
Epoch 392/500
4908/4908 [=====

4908/4908 [==============================] - 1s - loss: 0.3957     
Epoch 424/500
4908/4908 [==============================] - 1s - loss: 0.3957     
Epoch 425/500
4908/4908 [==============================] - 1s - loss: 0.3992     
Epoch 426/500
4908/4908 [==============================] - 1s - loss: 0.3981     
Epoch 427/500
4908/4908 [==============================] - 1s - loss: 0.3925     
Epoch 428/500
4908/4908 [==============================] - 1s - loss: 0.3987     
Epoch 429/500
4908/4908 [==============================] - 1s - loss: 0.3942     
Epoch 430/500
4908/4908 [==============================] - 1s - loss: 0.3873     
Epoch 431/500
4908/4908 [==============================] - 1s - loss: 0.4009     
Epoch 432/500
4908/4908 [==============================] - 1s - loss: 0.3869     
Epoch 433/500
4908/4908 [==============================] - 1s - loss: 0.4118     
Epoch 434/500
4908/4908 [==============================] - 1s - loss: 0.3891     
Epoch 435/500
4908/4908 [=====

4908/4908 [==============================] - 1s - loss: 0.3807     
Epoch 467/500
4908/4908 [==============================] - 1s - loss: 0.3923     
Epoch 468/500
4908/4908 [==============================] - 1s - loss: 0.3909     
Epoch 469/500
4908/4908 [==============================] - 1s - loss: 0.3840     
Epoch 470/500
4908/4908 [==============================] - 1s - loss: 0.3955     
Epoch 471/500
4908/4908 [==============================] - 1s - loss: 0.4003     
Epoch 472/500
4908/4908 [==============================] - 1s - loss: 0.4085     
Epoch 473/500
4908/4908 [==============================] - 1s - loss: 0.3891     
Epoch 474/500
4908/4908 [==============================] - 1s - loss: 0.3966     
Epoch 475/500
4908/4908 [==============================] - 1s - loss: 0.3767     
Epoch 476/500
4908/4908 [==============================] - 1s - loss: 0.3856     
Epoch 477/500
4908/4908 [==============================] - 1s - loss: 0.3908     
Epoch 478/500
4908/4908 [=====

In [100]:
generative_model.save('Text_gen_01_words_no_embedding')

In [101]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 30
dataX = []
dataY = []
split_text = raw_text.split()
for i in range(0, len(words) - seq_length, 1):
    seq_in = split_text[i:i + seq_length]
    seq_out = split_text[i + seq_length]
    dataX.append([word_to_int[word] for word in seq_in])
    dataY.append(word_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  4908


In [102]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]


In [103]:
## Create a seed from scratch

In [143]:
sam_text = "And a then Alice looked at the cat and the mad hatter and said 'how dare you rabbit' cried alice again, for this time the mouse was bristling all over every "
len(sam_text.split())
print(sam_text.lower().split())


['and', 'a', 'then', 'alice', 'looked', 'at', 'the', 'cat', 'and', 'the', 'mad', 'hatter', 'and', 'said', "'how", 'dare', 'you', "rabbit'", 'cried', 'alice', 'again,', 'for', 'this', 'time', 'the', 'mouse', 'was', 'bristling', 'all', 'over', 'every']


In [144]:
sam_pattern= []
for word in sam_text.lower().split():
    lookup = word_to_int[word]
    sam_pattern.append(lookup)
    
print(len(sam_pattern))

31


In [145]:
sam_pattern

[1,
 3,
 56,
 14,
 80,
 16,
 0,
 229,
 1,
 0,
 580,
 152,
 1,
 6,
 222,
 626,
 10,
 3208,
 200,
 14,
 112,
 23,
 31,
 75,
 0,
 179,
 9,
 3144,
 19,
 117,
 295]

In [140]:
print(pattern)
print("\"", ' '.join([int_to_word[value] for value in pattern]), "\"")

[111, 12, 4, 646, 2794, 35, 8, 0, 1608, 63, 4, 3266, 2749, 24, 1523, 61, 0, 863, 4720, 15, 1892, 1, 3331, 15, 3661, 4, 47, 87, 3, 658]
" herself, as she swam lazily about in the pool, 'and she sits purring so nicely by the fire, licking her paws and washing her face--and she is such a nice "


In [81]:
generated_text = []

# generate characters
for i in range(100):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab_words)
    prediction = generative_model.predict(x, verbose=0)
    index = np.argmax(prediction)
    #print(index)
    result = int_to_word[index]
    seq_in = [int_to_word[value] for value in pattern]
    #print(result)
    pattern.append(index)
    generated_text.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")


Done.


In [82]:
print(pattern)
print("\"", ' '.join([int_to_word[value] for value in pattern]), "\"")
print("\"", ' '.join([int_to_word[value] for value in generated_text]), "\"")

[186, 221, 325, 5, 30, 4339, 1, 647, 4315, 1, 4104, 1, 154, 0, 3, 1235, 2, 1494, 1, 0, 427, 791, 0, 11, 885, 1425, 11, 115, 309, 2]
" 'i'm sure those of not soft and generally hung and pegs. and took the a struck to dog and the house (alice the i indeed change, i can glad to "
" felt fallen she it: i was a duck and a dodo, she lory can't was eaglet, and several other curious creatures. and led and way, and the whole party swam the the shore. chapter iii. and caucus-race and a french tale the australia?' (and the tried capital curtsey i she think!' (dinah the the cat.) 'i hope they'll remember the saucer with milk and tea-time. dinah the foot, smiling jaws!' 'i'm sure those of not soft and generally hung and pegs. and took the a struck to dog and the house (alice the i indeed change, i can glad to "


In [81]:
generated_text = []

# generate characters
for i in range(100):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab_words)
    prediction = generative_model.predict(x, verbose=0)
    index = np.argmax(prediction)
    #print(index)
    result = int_to_word[index]
    seq_in = [int_to_word[value] for value in pattern]
    #print(result)
    pattern.append(index)
    generated_text.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")


Done.


In [82]:
print(pattern)
print("\"", ' '.join([int_to_word[value] for value in pattern]), "\"")
print("\"", ' '.join([int_to_word[value] for value in generated_text]), "\"")

[186, 221, 325, 5, 30, 4339, 1, 647, 4315, 1, 4104, 1, 154, 0, 3, 1235, 2, 1494, 1, 0, 427, 791, 0, 11, 885, 1425, 11, 115, 309, 2]
" 'i'm sure those of not soft and generally hung and pegs. and took the a struck to dog and the house (alice the i indeed change, i can glad to "
" felt fallen she it: i was a duck and a dodo, she lory can't was eaglet, and several other curious creatures. and led and way, and the whole party swam the the shore. chapter iii. and caucus-race and a french tale the australia?' (and the tried capital curtsey i she think!' (dinah the the cat.) 'i hope they'll remember the saucer with milk and tea-time. dinah the foot, smiling jaws!' 'i'm sure those of not soft and generally hung and pegs. and took the a struck to dog and the house (alice the i indeed change, i can glad to "


## Overfitting ?

In [108]:
generated_text = []

# generate characters
for i in range(100):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab_words)
    prediction = generative_model.predict(x, verbose=0)
    index = np.argmax(prediction)
    #print(index)
    result = int_to_word[index]
    seq_in = [int_to_word[value] for value in pattern]
    #print(result)
    pattern.append(index)
    generated_text.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")


Done.


In [109]:
print(pattern)
print("\"", ' '.join([int_to_word[value] for value in pattern]), "\"")
print("\"", ' '.join([int_to_word[value] for value in generated_text]), "\"")

[111, 12, 4, 646, 2794, 35, 8, 0, 1608, 63, 4, 3266, 2749, 24, 1523, 61, 0, 863, 4720, 15, 1892, 1, 3331, 15, 3661, 4, 47, 87, 3, 658]
" herself, as she swam lazily about in the pool, 'and she sits purring so nicely by the fire, licking her paws and washing her face--and she is such a nice "
" 'not like cats!' cried the mouse, in a shrill, passionate voice. 'would you like cats if you were me?' 'well, perhaps not,' said alice in a soothing tone: 'don't be angry about it. and yet i wish i could show you our cat dinah: i think you'd take a fancy to cats if you could only see her. she is such a dear quiet thing,' alice went on, half to herself, as she swam lazily about in the pool, 'and she sits purring so nicely by the fire, licking her paws and washing her face--and she is such a nice "


In [146]:
print(sam_pattern)
print("\"", ' '.join([int_to_word[value] for value in sam_pattern]), "\"")

[1, 3, 56, 14, 80, 16, 0, 229, 1, 0, 580, 152, 1, 6, 222, 626, 10, 3208, 200, 14, 112, 23, 31, 75, 0, 179, 9, 3144, 19, 117, 295]
" and a then alice looked at the cat and the mad hatter and said 'how dare you rabbit' cried alice again, for this time the mouse was bristling all over every "


In [147]:
#sam_pattern.append(0)
generated_text = []

# generate characters
for i in range(100):
    x = np.reshape(sam_pattern, (1, len(sam_pattern), 1))
    x = x / float(n_vocab_words)
    prediction = generative_model.predict(x, verbose=0)
    index = np.argmax(prediction)
    #print(index)
    result = int_to_word[index]
    seq_in = [int_to_word[value] for value in pattern]
    #print(result)
    sam_pattern.append(index)
    generated_text.append(index)
    sam_pattern = pattern[1:len(pattern)]
print("\nDone.")

ValueError: Error when checking : expected input_2 to have shape (None, 30, 1) but got array with shape (1, 31, 1)

In [113]:
print(pattern)
print("\"", ' '.join([int_to_word[value] for value in pattern]), "\"")
print("\"", ' '.join([int_to_word[value] for value in generated_text]), "\"")

[111, 12, 4, 646, 2794, 35, 8, 0, 1608, 63, 4, 3266, 2749, 24, 1523, 61, 0, 863, 4720, 15, 1892, 1, 3331, 15, 3661, 4, 47, 87, 3, 658]
" herself, as she swam lazily about in the pool, 'and she sits purring so nicely by the fire, licking her paws and washing her face--and she is such a nice "
"  "
